# Final project: BiCl2, BiS2, BiSCl

#### 1.relax the structures (cell and atomic positions) of MX2, MY2, and MXY

In [1]:
%%writefile finalProject1.py
from ase.io import read, write
from gpaw import GPAW, PW
from ase.optimize import BFGS
from ase.constraints import UnitCellFilter
import pickle
from ase.parallel import paropen

energies_slab = {}
for name in ['BiCl2', 'BiS2', 'BiSCl']:
    structures = name+'.xyz'
    structure = read(structures)
    
    calc = GPAW(mode=PW(500),
                xc='RPBE',
                kpts={'density': 3, 'gamma': True},
                poissonsolver={'dipolelayer': 'xy'},
                txt=name+'.out',
                )
    structure.pbc = (True, True, False)
    structure.set_calculator(calc)
    unitcell = UnitCellFilter(structure)
    relax = BFGS(unitcell, trajectory='relax--slab-'+name+'.traj', logfile='relax-slab-'+name+'.log')
    relax.run(fmax=0.1)
    energies_slab[name] = structure.get_potential_energy()
    calc.write('relax-'+name+'.gpw')  
    
#save
with paropen('energies_slab.pckl', 'wb') as f:  #binary protocols output
    pickle.dump(energies_slab, f)
    
with paropen('energies_slab.txt', 'w') as f:  # formatting output
    for x, y in energies_slab.items():
        print('energy-{0:6}: {1:6.3f}'.format(x, y), file=f)

Writing finalProject1.py


In [2]:
!bsub.py -t 1 -p 20 finalProject1.py

Job <8423722> is submitted to queue <hpc>.


#### 2.relax and calculate the adsorption of H, OH, Li, and Mg on the different inequivalent adsorption sites in a 3x3 supercell for the structures of point 1 (3x3 should allow to avoid interaction between the adsorbates) - you should relax only the positions of the atoms and not the cell.

In [3]:
%%writefile finalProject2.py
from ase.io import read, write
from ase.build import add_adsorbate
from ase import Atoms, Atom
from ase.build import molecule
from gpaw import GPAW, PW
from ase.optimize import BFGS
import numpy as np
import pickle
from ase.parallel import paropen

energies = {}
for system in ['BiCl2', 'BiS2', 'BiSCl']:
    for moleculas in ['H', 'OH', 'Li', 'Mg']:
        slab = read('relax--slab-'+system+'.traj') #equil to read('relax-BiClI.xyz.traj', -1)
        #print(slab)
        slab = slab.repeat((3, 3, 1))
        #print("supercell:", slab)        
        top_site = slab[13].position
        bridge_site = (0.5 * (slab[13].x + slab[10].x), 0.5 * (slab[13].y + slab[10].y), slab[13].z)
        bridge_site = np.asarray(bridge_site) #transfer type of variable to array
        hollow_site = (1/3. * (slab[13].x + slab[10].x + slab[1].x), 1/3. * (slab[13].y + slab[10].y + slab[1].y), slab[13].z)
        hollow_site = np.asarray(hollow_site) #transfer type of variable to array
        for sites in ['top', 'bridge', 'hollow']:
            if sites == 'top':
                position = top_site
            elif sites == 'bridge':
                position = bridge_site
            elif sites == 'hollow':
                position = hollow_site
            
            name = system+'-'+moleculas+'-'+sites
            calc = GPAW(mode=PW(500),
                        xc='RPBE',
                        #kpts={'size': (5, 5, 1), 'gamma': True},
                        kpts={'density': 3, 'gamma': True},  
                        poissonsolver={'dipolelayer': 'xy'},
                        txt='out-'+name+'.out',
                        )
            
            ads = 0
            if moleculas == 'H':
                #add adsorbate
                ads = Atoms([Atom('H', (0., 0., 0.))])
                ads.translate(position + (0., 0., 1.5))  #Translate atomic positions, 13 -> top site                        
                
            if moleculas == 'OH':
                #add adsorbate
                ads = Atoms([Atom('O', (0., 0., 0.)),
                             Atom('H', (0., 0., 1.))])
                ads.translate(position + (0., 0., 1.5)) 
            
            if moleculas == 'Li':
                #add adsorbate
                ads = Atoms([Atom('Li', (0., 0., 0.))])
                ads.translate(position + (0., 0., 1.5)) 
            
            if moleculas == 'Mg':
                #add adsorbate
                ads = Atoms([Atom('Mg', (0., 0., 0.))])
                ads.translate(position + (0., 0., 1.5))

            slab_c = slab.copy()
            slab_c.extend(ads)
            #relax
            slab_c.pbc = (True, True, False)
            slab_c.set_calculator(calc)
            relax_c = BFGS(slab_c, trajectory='relax--'+name+'.traj', logfile='log-relax-'+name+'.log')
            relax_c.run(fmax=0.1)
            energies[name] = slab_c.get_potential_energy()
            calc.write('gpw-relax-'+name+'.gpw')
        
        with paropen('energies.pckl', 'wb') as f:  #binary protocols output
            pickle.dump(energies, f)

#another side sites for BiSCl
slab = read('relax--slab-BiSCl.traj') 
slab = slab.repeat((3, 3, 1))
top_back = slab[14].position
bridge_back = (0.5 * (slab[14].x + slab[26].x), 0.5 * (slab[14].y + slab[26].y), slab[14].z)
bridge_back = np.asarray(bridge_back) #transfer type of variable to array
hollow_back = (1/3. * (slab[14].x + slab[26].x + slab[17].x), 1/3. * (slab[14].y + slab[26].y + slab[17].y), slab[14].z)
hollow_back = np.asarray(hollow_back) #transfer type of variable to array
for moleculas in ['H', 'OH', 'Li', 'Mg']:
    for sites in ['top', 'bridge', 'hollow']:
        if sites == 'top':
            position = top_back
        elif sites == 'bridge':
            position = bridge_back
        elif sites == 'hollow':
            position = hollow_back
        
        name = 'BiSCl-'+moleculas+'-'+sites+'-back'
        calc = GPAW(mode=PW(500),
                xc='RPBE',
                #kpts={'size': (5, 5, 1), 'gamma': True},
                kpts={'density': 3, 'gamma': True}, 
                poissonsolver={'dipolelayer': 'xy'},
                txt='out-'+name+'.out',
                )
        
        ads = 0
        if moleculas == 'H':
            #add adsorbate
            ads = Atoms([Atom('H', (0., 0., 0.))])
            ads.translate(position + (0., 0., -1.5)) 
                
        elif moleculas == 'OH':
            #add adsorbate
            ads = Atoms([Atom('O', (0., 0., 0.)),
                         Atom('H', (0., 0., -1.))])
            ads.translate(position + (0., 0., -1.5))  
            
        elif moleculas == 'Li':
            #add adsorbate
            ads = Atoms([Atom('Li', (0., 0., 0.))])
            ads.translate(position + (0., 0., -1.5)) 
            
        elif moleculas == 'Mg':
            #add adsorbate
            ads = Atoms([Atom('Mg', (0., 0., 0.))])
            ads.translate(position + (0., 0., -1.5)) 
        
        slab_c = slab.copy()
        slab_c.extend(ads)
#         from ase.visualize import view
#         view(slab_c)
#         assert False
        #relax
        slab_c.pbc = (True, True, False)
        slab_c.set_calculator(calc)
        relax_c = BFGS(slab_c, trajectory='relax--'+name+'.traj', logfile='log-relax-'+name+'.log')
        relax_c.run(fmax=0.1)
        energies[name] = slab_c.get_potential_energy()
        calc.write('gpw-relax-'+name+'.gpw')
    
    with paropen('energies.pckl', 'wb') as f:  #binary protocols output
        pickle.dump(energies, f)

#save
with paropen('energies.pckl', 'wb') as f:  #binary protocols output
    pickle.dump(energies, f)
    
with paropen('energies.txt', 'w') as f:  # formatting output
    for x, y in energies.items():
        print('energy-{0:6}: {1:6.3f}'.format(x, y), file=f)

print('end............')

Overwriting finalProject2.py


In [5]:
!bsub.py -t 3 -p 20 finalProject2.py

#### 3.add strain (+/- 2% compressive and tensile strain) both in the zigzag and armchair direction and calculate the adsorptions

#### add -2% compressive strain both in the zigzag and armchair direction and calculate the adsorptions

In [6]:
%%writefile finalProject3.py
from ase.io import read, write
from ase.build import add_adsorbate
from ase import Atoms, Atom
from ase.build import molecule
from gpaw import GPAW, PW
from ase.optimize import BFGS
import numpy as np
import pickle
from ase.parallel import paropen

energies_compressive = {}
for system in ['BiCl2', 'BiS2', 'BiSCl']:
    for moleculas in ['H', 'OH', 'Li', 'Mg']:
        slab = read('relax--slab-'+system+'.traj') 
        slab = slab.repeat((3, 3, 1))
#         print('before:', slab.get_cell())
#         from ase.visualize import view
#         view(slab)        
        slab.set_cell(slab.get_cell()*[0.98, 0.98, 1],scale_atoms=True)  #add strain
#         from ase.visualize import view
#         view(slab)
#         assert False
        top_site = slab[13].position
        bridge_site = (0.5 * (slab[13].x + slab[10].x), 0.5 * (slab[13].y + slab[10].y), slab[13].z)
        bridge_site = np.asarray(bridge_site) #transfer type of variable to array
        hollow_site = (1/3. * (slab[13].x + slab[10].x + slab[1].x), 1/3. * (slab[13].y + slab[10].y + slab[1].y), slab[13].z)
        hollow_site = np.asarray(hollow_site) #transfer type of variable to array
        for sites in ['top', 'bridge', 'hollow']:
            if sites == 'top':
                position = top_site
            elif sites == 'bridge':
                position = bridge_site
            elif sites == 'hollow':
                position = hollow_site
            
            name = system+'-'+moleculas+'-'+sites
            calc = GPAW(mode=PW(500),
                        xc='RPBE',
                        #kpts={'size': (5, 5, 1), 'gamma': True},
                        kpts={'density': 3, 'gamma': True},  
                        poissonsolver={'dipolelayer': 'xy'},
                        txt='out-'+name+'.out',
                        )
            
            ads = 0
            if moleculas == 'H':
                #add adsorbate
                ads = Atoms([Atom('H', (0., 0., 0.))])
                ads.translate(position + (0., 0., 1.5))                  
                
            if moleculas == 'OH':
                #add adsorbate
                ads = Atoms([Atom('O', (0., 0., 0.)),
                             Atom('H', (0., 0., 1.))])
                ads.translate(position + (0., 0., 1.5)) 
            
            if moleculas == 'Li':
                #add adsorbate
                ads = Atoms([Atom('Li', (0., 0., 0.))])
                ads.translate(position + (0., 0., 1.5)) 
            
            if moleculas == 'Mg':
                #add adsorbate
                ads = Atoms([Atom('Mg', (0., 0., 0.))])
                ads.translate(position + (0., 0., 1.5))

            slab_c = slab.copy()
            slab_c.extend(ads)
#             from ase.visualize import view
#             view(slab_c)
#             assert False
            #relax
            slab_c.pbc = (True, True, False)
            slab_c.set_calculator(calc)
            relax_c = BFGS(slab_c, trajectory='relax--'+name+'.traj', logfile='log-relax-'+name+'.log')
            relax_c.run(fmax=0.1)
            energies_compressive[name] = slab_c.get_potential_energy()
            calc.write('gpw-relax-'+name+'.gpw')
        
        with paropen('energies_compressive.pckl', 'wb') as f:  #binary protocols output
            pickle.dump(energies_compressive, f)

#another side sites for BiSCl
slab = read('relax--slab-BiSCl.traj') 
slab = slab.repeat((3, 3, 1))
slab.set_cell(slab.get_cell()*[0.98, 0.98, 1],scale_atoms=True)  #add strain
top_back = slab[14].position
bridge_back = (0.5 * (slab[14].x + slab[26].x), 0.5 * (slab[14].y + slab[26].y), slab[14].z)
bridge_back = np.asarray(bridge_back) #transfer type of variable to array
hollow_back = (1/3. * (slab[14].x + slab[26].x + slab[17].x), 1/3. * (slab[14].y + slab[26].y + slab[17].y), slab[14].z)
hollow_back = np.asarray(hollow_back) #transfer type of variable to array
for moleculas in ['H', 'OH', 'Li', 'Mg']:
    for sites in ['top', 'bridge', 'hollow']:
        if sites == 'top':
            position = top_back
        elif sites == 'bridge':
            position = bridge_back
        elif sites == 'hollow':
            position = hollow_back
        
        name = 'BiSCl-'+moleculas+'-'+sites+'-back'
        calc = GPAW(mode=PW(500),
                xc='RPBE',
                #kpts={'size': (5, 5, 1), 'gamma': True},
                kpts={'density': 3, 'gamma': True}, 
                poissonsolver={'dipolelayer': 'xy'},
                txt='out-'+name+'.out',
                )
        
        ads = 0
        if moleculas == 'H':
            #add adsorbate
            ads = Atoms([Atom('H', (0., 0., 0.))])
            ads.translate(position + (0., 0., -1.5)) 
                
        elif moleculas == 'OH':
            #add adsorbate
            ads = Atoms([Atom('O', (0., 0., 0.)),
                         Atom('H', (0., 0., -1.))])
            ads.translate(position + (0., 0., -1.5))  
            
        elif moleculas == 'Li':
            #add adsorbate
            ads = Atoms([Atom('Li', (0., 0., 0.))])
            ads.translate(position + (0., 0., -1.5)) 
            
        elif moleculas == 'Mg':
            #add adsorbate
            ads = Atoms([Atom('Mg', (0., 0., 0.))])
            ads.translate(position + (0., 0., -1.5)) 
        
        slab_c = slab.copy()
        slab_c.extend(ads)
#         from ase.visualize import view
#         view(slab_c)
#         assert False # Hack to break the execution by raising an error intentionally.
        slab_c.pbc = (True, True, False)
        slab_c.set_calculator(calc)
        relax_c = BFGS(slab_c, trajectory='relax--'+name+'.traj', logfile='log-relax-'+name+'.log')
        relax_c.run(fmax=0.1)
        energies_compressive[name] = slab_c.get_potential_energy()
        calc.write('gpw-relax-'+name+'.gpw')
    
    with paropen('energies_compressive.pckl', 'wb') as f:
        pickle.dump(energies_compressive, f)

#print(energies_compressive_slab)
with paropen('energies_compressive.pckl', 'wb') as f:  #binary protocols output
    pickle.dump(energies_compressive, f)
    
with paropen('energies_compressive.txt', 'w') as f:  # formatting output
    for x, y in energies_compressive.items():
        print('energy-{0:6}: {1:6.3f}'.format(x, y), file=f)

print('end............')

Writing finalProject3.py


#### add +2% tensile strain both in the zigzag and armchair direction and calculate the adsorptions

In [10]:
%%writefile finalProject3_2.py
from ase.io import read, write
from ase.build import add_adsorbate
from ase import Atoms, Atom
from ase.build import molecule
from gpaw import GPAW, PW
from ase.optimize import BFGS
import numpy as np
import pickle
from ase.parallel import paropen

energies_tensile = {}
for system in ['BiCl2', 'BiS2', 'BiSCl']:
    for moleculas in ['H', 'OH', 'Li', 'Mg']:
        slab = read('relax--slab-'+system+'.traj') 
        slab = slab.repeat((3, 3, 1))
#         print('before:', slab.get_cell())
#         from ase.visualize import view
#         view(slab)        
        slab.set_cell(slab.get_cell()*[1.02, 1.02, 1],scale_atoms=True)  #add strain
#         print('after:', slab.get_cell())
#         from ase.visualize import view
#         view(slab)
#         assert False
        top_site = slab[13].position
        bridge_site = (0.5 * (slab[13].x + slab[10].x), 0.5 * (slab[13].y + slab[10].y), slab[13].z)
        bridge_site = np.asarray(bridge_site) #transfer type of variable to array
        hollow_site = (1/3. * (slab[13].x + slab[10].x + slab[1].x), 1/3. * (slab[13].y + slab[10].y + slab[1].y), slab[13].z)
        hollow_site = np.asarray(hollow_site) #transfer type of variable to array
        for sites in ['top', 'bridge', 'hollow']:
            if sites == 'top':
                position = top_site
            elif sites == 'bridge':
                position = bridge_site
            elif sites == 'hollow':
                position = hollow_site
            
            name = system+'-'+moleculas+'-'+sites
            calc = GPAW(mode=PW(500),
                        xc='RPBE',
                        #kpts={'size': (5, 5, 1), 'gamma': True},
                        kpts={'density': 3, 'gamma': True},  
                        poissonsolver={'dipolelayer': 'xy'},
                        txt='out-'+name+'.out',
                        )
            
            ads = 0
            if moleculas == 'H':
                #add adsorbate
                ads = Atoms([Atom('H', (0., 0., 0.))])
                ads.translate(position + (0., 0., 1.5))                   
                
            if moleculas == 'OH':
                #add adsorbate
                ads = Atoms([Atom('O', (0., 0., 0.)),
                             Atom('H', (0., 0., 1.))])
                ads.translate(position + (0., 0., 1.5)) 
            
            if moleculas == 'Li':
                #add adsorbate
                ads = Atoms([Atom('Li', (0., 0., 0.))])
                ads.translate(position + (0., 0., 1.5)) 
            
            if moleculas == 'Mg':
                #add adsorbate
                ads = Atoms([Atom('Mg', (0., 0., 0.))])
                ads.translate(position + (0., 0., 1.5))

            slab_c = slab.copy()
            slab_c.extend(ads)
            #relax
            slab_c.pbc = (True, True, False)
            slab_c.set_calculator(calc)
            relax_c = BFGS(slab_c, trajectory='relax--'+name+'.traj', logfile='log-relax-'+name+'.log')
            relax_c.run(fmax=0.1)
            energies_tensile[name] = slab_c.get_potential_energy()
            calc.write('gpw-relax-'+name+'.gpw')
        
        with paropen('energies_tensile.pckl', 'wb') as f: 
            pickle.dump(energies_tensile, f)

#another side sites for BiSCl
slab = read('relax--slab-BiSCl.traj') 
slab = slab.repeat((3, 3, 1))
slab.set_cell(slab.get_cell()*[1.02, 1.02, 1],scale_atoms=True)  #add strain
top_back = slab[14].position
bridge_back = (0.5 * (slab[14].x + slab[26].x), 0.5 * (slab[14].y + slab[26].y), slab[14].z)
bridge_back = np.asarray(bridge_back) #transfer type of variable to array
hollow_back = (1/3. * (slab[14].x + slab[26].x + slab[17].x), 1/3. * (slab[14].y + slab[26].y + slab[17].y), slab[14].z)
hollow_back = np.asarray(hollow_back) #transfer type of variable to array
for moleculas in ['H', 'OH', 'Li', 'Mg']:
    for sites in ['top', 'bridge', 'hollow']:
        if sites == 'top':
            position = top_back
        elif sites == 'bridge':
            position = bridge_back
        elif sites == 'hollow':
            position = hollow_back
        
        name = 'BiSCl-'+moleculas+'-'+sites+'-back'
        calc = GPAW(mode=PW(500),
                xc='RPBE',
                #kpts={'size': (5, 5, 1), 'gamma': True},
                kpts={'density': 3, 'gamma': True}, 
                poissonsolver={'dipolelayer': 'xy'},
                txt='out-'+name+'.out',
                )
        
        ads = 0
        if moleculas == 'H':
            #add adsorbate
            ads = Atoms([Atom('H', (0., 0., 0.))])
            ads.translate(position + (0., 0., -1.5)) 
                
        elif moleculas == 'OH':
            #add adsorbate
            ads = Atoms([Atom('O', (0., 0., 0.)),
                         Atom('H', (0., 0., -1.))])
            ads.translate(position + (0., 0., -1.5))  
            
        elif moleculas == 'Li':
            #add adsorbate
            ads = Atoms([Atom('Li', (0., 0., 0.))])
            ads.translate(position + (0., 0., -1.5)) 
            
        elif moleculas == 'Mg':
            #add adsorbate
            ads = Atoms([Atom('Mg', (0., 0., 0.))])
            ads.translate(position + (0., 0., -1.5)) 
        
        slab_c = slab.copy()
        slab_c.extend(ads)
#         from ase.visualize import view
#         view(slab_c)
#         assert False 
        #relax
        slab_c.pbc = (True, True, False)
        slab_c.set_calculator(calc)
        relax_c = BFGS(slab_c, trajectory='relax--'+name+'.traj', logfile='log-relax-'+name+'.log')
        relax_c.run(fmax=0.1)
        energies_tensile[name] = slab_c.get_potential_energy()
        calc.write('gpw-relax-'+name+'.gpw')
    
    with paropen('energies_tensile.pckl', 'wb') as f:
        pickle.dump(energies_tensile, f)

#print(energies_tensile_slab)
with paropen('energies_tensile.pckl', 'wb') as f:  #binary protocols output
    pickle.dump(energies_tensile, f)
    
with paropen('energies_tensile.txt', 'w') as f:  # formatting output
    for x, y in energies_tensile.items():
        print('energy-{0:6}: {1:6.3f}'.format(x, y), file=f)

print('end............')

Writing finalProject3_2.py


#### 4.(optional) because of embedded strain between the X and Y layer, the 2D Janus sheet tends to wrap into a nanotube (structure is given). Calculate the adsoption of H, OH, Li, and Mg on the inequivalent sites of the 1D inorganic tube.

In [33]:
%%writefile finalProject4-0.py
from ase.io import read, write
from gpaw import GPAW, PW
from ase.optimize import BFGS
from ase.constraints import UnitCellFilter
import pickle
from ase.parallel import paropen

energies_tube_slab = {}
structures = 'BiSCl_1D.xyz'
structure = read('structures')

calc = GPAW(mode=PW(500),
            xc='RPBE',
            kpts={'density': 3, 'gamma': True},
            #poissonsolver={'dipolelayer': 'xy'},
            txt='out-BiSCl_1D.out',
            )
structure.pbc = (False, False, True)
structure.set_calculator(calc)
#unitcell = UnitCellFilter(structure)
relax = BFGS(structure, trajectory='relax--slab-BiSCl_1D.traj', logfile='log-relax-slab-BiSCl_1D.log')
relax.run(fmax=0.1)
energies_tube_slab['BiSCl_1D'] = structure.get_potential_energy()
calc.write('gpw-relax-BiSCl_1D.gpw') 
    
#print(energies_slab)
with paropen('energies_tube_slab.pckl', 'wb') as f:  #binary protocols output
    pickle.dump(energies_tube_slab, f)
    
with paropen('energies_tube_slab.txt', 'w') as f:  # formatting output
    for x, y in energies_tube_slab.items():
        print('energy-{0:6}: {1:6.3f}'.format(x, y), file=f)

Overwriting finalProject4-0.py


In [8]:
!bsub.py -t 3 -p 20 finalProject4-0.py

In [9]:
%%writefile finalProject4.py
from ase.io import read, write
from ase.build import add_adsorbate
from ase import Atoms, Atom
from ase.build import molecule
from gpaw import GPAW, PW
from ase.optimize import BFGS
import numpy as np
import pickle
from ase.parallel import paropen

tube = read('relax--slab-BiSCl_1D.traj')
#tube = read('relax--slab-BiS2.traj')
slab = tube.repeat((1, 1, 2))
# slab = tube.repeat((1, 1, 3))
# from ase.visualize import view
# view(slab)
# assert False
top_site = slab[35].position
bridge_site = (0.5 * (slab[35].x + slab[83].x), 0.5 * (slab[35].y + slab[83].y), 0.5 * (slab[35].z + slab[83].z))
bridge_site = np.asarray(bridge_site) #transfer type of variable to array
hollow_site = (1/3. * (slab[35].x + slab[83].x + slab[84].x), 1/3. * (slab[35].y + slab[83].y + slab[84].y), 1/3. * (slab[35].z + slab[83].z + slab[84].z))
hollow_site = np.asarray(hollow_site) #transfer type of variable to array
energies_tube = {}
#outer adsorption
for moleculas in ['H', 'OH', 'Li', 'Mg']:
    for sites in ['top', 'bridge', 'hollow']:
        if sites == 'top':
            position = top_site
        elif sites == 'bridge':
            position = bridge_site
        elif sites == 'hollow':
            position = hollow_site

        name = 'BiSCl_1D-'+moleculas+'-'+sites
        calc = GPAW(mode=PW(500),
                    xc='RPBE',
                    #kpts={'size': (5, 5, 1), 'gamma': True},
                    kpts={'density': 3, 'gamma': True},  
                    #poissonsolver={'dipolelayer': 'xy'},
                    txt='out-'+name+'.out',
                    )

        ads = 0
        if moleculas == 'H':
            #add adsorbate
            ads = Atoms([Atom('H', (0., 0., 0.))])
            ads.translate(position + (0., 1.5, 0.))                     

        if moleculas == 'OH':
            #add adsorbate
            ads = Atoms([Atom('O', (0., 0., 0.)),
                         Atom('H', (0., 1., 0.))])
            ads.translate(position + (0., 1.5, 0.)) 

        if moleculas == 'Li':
            #add adsorbate
            ads = Atoms([Atom('Li', (0., 0., 0.))])
            ads.translate(position + (0., 1.5, 0.)) 

        if moleculas == 'Mg':
            #add adsorbate
            ads = Atoms([Atom('Mg', (0., 0., 0.))])
            ads.translate(position + (0., 1.5, 0.))

        slab_c = slab.copy()
        slab_c.extend(ads)
#         from ase.visualize import view
#         view(slab_c)
#         assert False
        #relax
        slab_c.pbc = (False, False, True)
        slab_c.set_calculator(calc)
        relax_c = BFGS(slab_c, trajectory='relax--'+name+'.traj', logfile='log-relax-'+name+'.log')
        relax_c.run(fmax=0.1)
        energies_tube[name] = slab_c.get_potential_energy()
        calc.write('gpw-relax-'+name+'.gpw')

    with paropen('energies_tube.pckl', 'wb') as f:  #binary protocols output
        pickle.dump(energies_tube, f)


#inner adsorption
top_back = slab[20].position
bridge_back = (0.5 * (slab[20].x + slab[68].x), 0.5 * (slab[20].y + slab[68].y), 0.5 * (slab[20].z + slab[68].z))
bridge_back = np.asarray(bridge_back) #transfer type of variable to array
hollow_back = (1/3. * (slab[20].x + slab[68].x + slab[19].x), 1/3. * (slab[20].y + slab[68].y + slab[19].y), 1/3. * (slab[20].z + slab[68].z + slab[19].z))
hollow_back = np.asarray(hollow_back) #transfer type of variable to array
for moleculas in ['H', 'OH', 'Li', 'Mg']:
    for sites in ['top', 'bridge', 'hollow']:
        if sites == 'top':
            position = top_back
        elif sites == 'bridge':
            position = bridge_back
        elif sites == 'hollow':
            position = hollow_back
        
        name = 'BiSCl_1D-'+moleculas+'-'+sites+'-inner'
        calc = GPAW(mode=PW(500),
                xc='RPBE',
                #kpts={'size': (5, 5, 1), 'gamma': True},
                kpts={'density': 3, 'gamma': True}, 
                #poissonsolver={'dipolelayer': 'xy'},
                txt='out-'+name+'.out',
                )
        
        ads = 0
        if moleculas == 'H':
            #add adsorbate
            ads = Atoms([Atom('H', (0., 0., 0.))])
            ads.translate(position + (0., -1.5, 0.))                     

        if moleculas == 'OH':
            #add adsorbate
            ads = Atoms([Atom('O', (0., 0., 0.)),
                         Atom('H', (0., -1., 0.))])
            ads.translate(position + (0., -1.5, 0.)) 

        if moleculas == 'Li':
            #add adsorbate
            ads = Atoms([Atom('Li', (0., 0., 0.))])
            ads.translate(position + (0., -1.5, 0.)) 

        if moleculas == 'Mg':
            #add adsorbate
            ads = Atoms([Atom('Mg', (0., 0., 0.))])
            ads.translate(position + (0., -1.5, 0.))
        
        slab_c = slab.copy()
        slab_c.extend(ads)
#         from ase.visualize import view
#         view(slab_c)
#         assert False
        #relax
        slab_c.pbc = (False, False, True)
        slab_c.set_calculator(calc)
        relax_c = BFGS(slab_c, trajectory='relax--'+name+'.traj', logfile='log-relax-'+name+'.log')
        relax_c.run(fmax=0.1)
        energies_tube[name] = slab_c.get_potential_energy()
        calc.write('gpw-relax-'+name+'.gpw')

    with paropen('energies_tube.pckl', 'wb') as f:  #binary protocols output
        pickle.dump(energies_tube, f)

#print(energies_tube)
with paropen('energies_tube.pckl', 'wb') as f:  #binary protocols output
    pickle.dump(energies_tube, f)
    
with paropen('energies_tube.txt', 'w') as f:  # formatting output
    for x, y in energies_tube.items():
        print('energy-{0:6}: {1:6.3f}'.format(x, y), file=f)


print('end............')


Overwriting finalProject4.py
